In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns # data visualisation
import matplotlib.pyplot as plt # data visualisation 
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

In [2]:
!pip install WordCloud


    python-dotenv (>=0.13.*) ; extra == 'standard'
                   ~~~~~~~^

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
df = pd.read_csv("spam.csv", encoding='latin-1')

In [4]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [6]:
df.rename({'v1':'Tag','v2':'Mail'},axis=1,inplace=True) #Renaming the Columns
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1) #Removing the Last Three Columns
df

,Tag,Mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [7]:
df.duplicated().sum() #checking duplicated values

df = df.drop_duplicates(keep='first') #remove duplicates
df.head()

,Tag,Mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df.shape

(5169, 2)

In [9]:
df.loc[:,'Tag']=df.Tag.map({'ham':0, 'spam':1}) #changing target column
df

C:\Users\dell\AppData\Local\Temp\ipykernel_9636\2474651114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'Tag']=df.Tag.map({'ham':0, 'spam':1}) #changing target column
C:\Users\dell\AppData\Local\Temp\ipykernel_9636\2474651114.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,'Tag']=df.Tag.map({'ham':0, 'spam':1}) #changing target column


,Tag,Mail
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [10]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

# Download the necessary NLTK datasets if not already done
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
ps = PorterStemmer()

def text_processing(text):
    if not isinstance(text, str):
        return ""  # Handle non-string input

    # Tokenize and convert to lowercase
    text = nltk.word_tokenize(text.lower())
    y = []
    
    # Keep only alphanumeric words
    for i in text:
        if i.isalnum():
            y.append(i)
    
    # Clear the text list for further processing
    text = y[:]
    y.clear()

    # Remove stopwords and punctuation, and apply stemming
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            stemming = ps.stem(i)
            y.append(stemming)
    
    return " ".join(y)


In [13]:
# ps = PorterStemmer()
# def text_processing(text):
#     text = nltk.word_tokenize(text.lower())
#     y = []
#     for i in text:
#         if i.isalnum():
#             y.append(i)
#     text = y[:]
#     y.clear()
    
#     for i in text:
#         if i not in stopwords.words('english') and i not in string.punctuation:
#             stemming = ps.stem(i)
#             y.append(stemming)
            
#     return " ".join(y)

In [14]:
print(df.columns)

Index(['Tag', 'Mail'], dtype='object')


In [15]:

df['transformed_mail'] = df['Mail'].apply(text_processing)


C:\Users\dell\AppData\Local\Temp\ipykernel_9636\402525720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['transformed_mail'] = df['Mail'].apply(text_processing)


In [16]:
df.head()

,Tag,Mail,transformed_mail
0,0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,0,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,0,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,0,"Nah I don't think he goes to usf, he lives aro...",nah think goe usf live around though


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score

In [18]:
tf = TfidfVectorizer(stop_words='english', lowercase=True, max_features=1000)
X = tf.fit_transform(df['transformed_mail']).toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
import joblib

# Assuming your model is named `rf_model`
model_save_path = 'tfidf_vectorizer.pkl'
joblib.dump(tf, model_save_path)

['tfidf_vectorizer.pkl']

In [20]:
y = np.array(df['Tag'])
y

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=48,test_size=0.2)

In [22]:
def train_evaluate_model(model, X_train, y_train, X_test, y_test, model_name=''):
    model.fit(X_train, y_train)
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    
# Calculate metrics
    train_accuracy = accuracy_score(y_train, train_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)
    train_precision = precision_score(y_train, train_predictions)
    test_precision = precision_score(y_test, test_predictions)
    train_recall = recall_score(y_train, train_predictions)
    test_recall = recall_score(y_test, test_predictions)
    

# Print metrics
    print(f"{model_name} - Training Metrics: Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")
    print(f"{model_name} - Test Metrics: Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}")
    print("----------------------------------------------------------------------------------")

In [23]:
models = [
    (LogisticRegression(), "Logistic Regression"),
    (MultinomialNB(), "Naive Bayes"),
     (RandomForestClassifier(), "Random Forest"),
    (KNeighborsClassifier(), "K-Nearest Neighbors"),
    (DecisionTreeClassifier(), "Decision Tree"),
    (LinearSVC(), "SVC")
]
for model, name in models:
    train_evaluate_model(model, X_train, y_train, X_test, y_test, model_name=name)

Logistic Regression - Training Metrics: Accuracy: 0.9724, Precision: 0.9904, Recall: 0.7889
Logistic Regression - Test Metrics: Accuracy: 0.9574, Precision: 0.9783, Recall: 0.6818
----------------------------------------------------------------------------------
Naive Bayes - Training Metrics: Accuracy: 0.9814, Precision: 0.9955, Recall: 0.8560
Naive Bayes - Test Metrics: Accuracy: 0.9700, Precision: 0.9550, Recall: 0.8030
----------------------------------------------------------------------------------
Random Forest - Training Metrics: Accuracy: 0.9993, Precision: 1.0000, Recall: 0.9942
Random Forest - Test Metrics: Accuracy: 0.9681, Precision: 0.9714, Recall: 0.7727
----------------------------------------------------------------------------------
K-Nearest Neighbors - Training Metrics: Accuracy: 0.9366, Precision: 1.0000, Recall: 0.4971
K-Nearest Neighbors - Test Metrics: Accuracy: 0.9188, Precision: 0.9800, Recall: 0.3712
-----------------------------------------------------------

c:\Users\dell\anaconda3\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [24]:
model = LinearSVC()
model.fit(X_train, y_train)
def predict_Mail(model, message, vectorizer):
    input_features = vectorizer.transform([message])
    prediction = model.predict(input_features)
    if prediction == 1:
        return "Spam" 
    if prediction == 0:
        return "Ham"

c:\Users\dell\anaconda3\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [25]:
test_mail1 ="Hi Paramjit This month's update is all about helping you transform your content.Creating something beautiful and engaging shouldn't be so hard. Our mission is to make it effortless.Please read about these new features below, try them out, and let us know what you think! And don't forget to subscribe to our YouTube channel for more Gamma content!"
print(f" classified as {predict_Mail(model, test_mail1, tf)}")

 classified as Ham


In [26]:
import joblib

# Assuming your model is named `rf_model`
model_save_path = 'spam_classifier_model.pkl'
joblib.dump(model, model_save_path)

['spam_classifier_model.pkl']